In [242]:
import pandas as pd
import numpy as np
import sys
import importlib.util
import os

Read file input.txt with path to data (.csv), operations, and percentace list

In [243]:
with open("input.txt", "r") as file:
    lines = [line.strip() for line in file if not line.lstrip().startswith("#")]

lines

['happynes_gauss.csv',
 '.',
 'implications.py',
 'Fodora',
 'operations.py',
 'T_F',
 '0, 1, 2, 1, 2, 0']

In [244]:

module_path = lines[1]
if os.path.exists(module_path):
    sys.path.append(module_path)
else:
    raise FileNotFoundError(f"Path {module_path} does not exist!")
module_name =  lines[2][0:-3]
imp = importlib.import_module(module_name)
module_name =  lines[4][0:-3]
ope = importlib.import_module(module_name)

In [245]:
data = pd.read_csv(lines[0],sep=";")
data

,Explained by: GDP per capita,Explained by: Social support,Explained by: Healthy life expectancy,Explained by: Freedom to make life choices,Explained by: Generosity,Explained by: Perceptions of corruption,Happiness score
0,0.856496,0.953030,0.822718,0.994595,0.232906,0.909710,1.000000
1,0.884110,0.941667,0.824841,0.971622,0.401709,0.906303,0.965848
2,0.876415,1.000000,0.852442,0.970270,0.576923,0.325383,0.951265
3,0.917157,0.928788,0.872611,0.914865,0.314103,0.785349,0.942957
4,0.880489,0.913636,0.835456,0.879730,0.579060,0.713799,0.925051
...,...,...,...,...,...,...,...
140,0.379810,0.642424,0.000000,0.566216,0.162393,0.030664,0.204541
141,0.680398,0.617424,0.297240,0.771622,0.025641,0.173765,0.196972
142,0.355364,0.100758,0.490446,0.839189,0.399573,0.926746,0.159498
143,0.428701,0.522727,0.286624,0.444595,0.226496,0.178876,0.109101


In [246]:
headers = list(data)
headers, header_to_compare = headers[:-1], headers[-1]
headers

['Explained by: GDP per capita',
 'Explained by: Social support',
 'Explained by: Healthy life expectancy',
 'Explained by: Freedom to make life choices',
 'Explained by: Generosity',
 'Explained by: Perceptions of corruption']

In [247]:
shape = (data.shape[0], data.shape[1]-1)

A = (data.iloc[:,:shape[1]])
b = (data.iloc[:,shape[1]])

In [248]:
dir(imp)[0:4], "..."

(['Einsteina', 'Fodora', 'Godela', 'Goguena'], '...')

In [249]:
u=[]
matrix_solution = matrix_solution = np.zeros(shape)

for i in range(shape[0]):
    for j in range(shape[1]):
        matrix_solution[i, j] = imp.implication_function(lines[3], A.iloc[i, j], b[j])
for j in range(shape[1]):
    u.append(min(matrix_solution[:, j]))
print("the greatest sulution Ax<=b\nu^T",u)

the greatest sulution Ax<=b
u^T [1.0, 0.96584825549197, 0.951264537566919, 0.942957356470371, 0.925050766106701, 0.923020121838656]


### Calculate the minimum with each column omitted:
- for each column has to be defined percent of indexes to remove
- if none then 0%
- solution is matrix_solution

In [250]:
num_of_headers = len(headers)
num_of_headers

6

In [251]:
percentage_for_each_column= list(map(int, lines[6].split(", "))) # %
indexes_to_remove = set()
for j in range(shape[1]):
    indexes_to_remove.update(A.iloc[:,j].sort_values(ascending=False).index[0:int(shape[0]*percentage_for_each_column[j]/100)])
len(indexes_to_remove)

6

In [252]:
# percentage_for_each_column= list(map(int, lines[6].split(", "))) # %
# indexes_to_remove = set()
# for j in range(shape[1]):
#     indexes_to_remove.update(A.iloc[:,j].sort_values(ascending=True).index[0:int(shape[0]*percentage_for_each_column[j]/100)])
# len(indexes_to_remove)

In [253]:
A_submatrix = A.drop(axis='index', index=list(indexes_to_remove)).copy()
b_submatrix = b.drop(axis='index', index=list(indexes_to_remove)).copy()
A_submatrix = A_submatrix.reset_index(drop=True)
b_submatrix = b_submatrix.reset_index(drop=True)
#A_submatrix
#b_submatrix

In [264]:
u=[]
shape_submatrix = A_submatrix.shape
matrix_solution = np.zeros(shape_submatrix)
for i in range(shape_submatrix[0]):
    for j in range(shape_submatrix[1]):
        matrix_solution[i, j] = imp.implication_function(lines[3], A_submatrix.iloc[i, j], b_submatrix[j])
for j in range(shape[1]):
    u.append(min(matrix_solution[:, j]))
print("the greatest sulution A(submatrix)x<=b(submatrix)\nu^T=",u)


the greatest sulution A(submatrix)x<=b(submatrix)
u^T= [1.0, 0.96584825549197, 1.0, 0.925050766106701, 1.0, 0.919328041351302]


In [275]:
b_to_check = np.zeros((shape[0],1))
for i in range(shape[0]): 
    wsolution_row = []
    for j in range(shape[1]): 
        wsolution_row.append(ope.operation_function(lines[5],A.iloc[i,j], u[j]))
    wsolution = max(wsolution_row)
    b_to_check[i,0] = wsolution
print(b_to_check)

[[0.9530303 ]
 [0.94166667]
 [0.96584826]
 [0.92878788]
 [0.91363636]
 [1.        ]
 [0.92505077]
 [0.93863636]
 [0.925     ]
 [0.93560606]
 [0.88257576]
 [0.91351351]
 [0.96378452]
 [0.87098234]
 [0.9       ]
 [0.89723857]
 [0.86590909]
 [0.95454545]
 [0.86328656]
 [0.92348485]
 [0.92505077]
 [0.94621212]
 [0.89324324]
 [0.92505077]
 [0.87972973]
 [0.85875962]
 [0.97283839]
 [0.81756757]
 [0.91742424]
 [0.89864865]
 [0.85031847]
 [0.80945946]
 [0.91756757]
 [0.91212121]
 [0.93333333]
 [0.93333333]
 [0.83863636]
 [0.79090909]
 [0.89459459]
 [0.92424242]
 [0.86942675]
 [0.925     ]
 [0.82272727]
 [0.81818182]
 [0.83378378]
 [0.83918919]
 [0.89621212]
 [0.88939394]
 [0.9       ]
 [0.86192847]
 [0.93409091]
 [0.84545455]
 [0.92505077]
 [0.91932059]
 [0.78648649]
 [0.88513514]
 [0.83484848]
 [0.82165605]
 [0.89278132]
 [0.91621622]
 [0.83378378]
 [0.76363636]
 [0.79166667]
 [0.92505077]
 [0.87651515]
 [0.76433121]
 [0.80909091]
 [0.90681818]
 [0.81891892]
 [0.88378378]
 [0.8472973 ]
 [0.88

In [266]:
#b
data[header_to_compare]

0      1.000000
1      0.965848
2      0.951265
3      0.942957
4      0.925051
         ...   
140    0.204541
141    0.196972
142    0.159498
143    0.109101
144    0.101717
Name: Happiness score, Length: 145, dtype: float64

In [267]:
# b_to_check = A*u
b_to_check

array([[0.9530303 ],
       [0.94166667],
       [0.96584826],
       [0.92878788],
       [0.91363636],
       [1.        ],
       [0.92505077],
       [0.93863636],
       [0.925     ],
       [0.93560606],
       [0.88257576],
       [0.91351351],
       [0.96378452],
       [0.87098234],
       [0.9       ],
       [0.89723857],
       [0.86590909],
       [0.95454545],
       [0.86328656],
       [0.92348485],
       [0.92505077],
       [0.94621212],
       [0.89324324],
       [0.92505077],
       [0.87972973],
       [0.85875962],
       [0.97283839],
       [0.81756757],
       [0.91742424],
       [0.89864865],
       [0.85031847],
       [0.80945946],
       [0.91756757],
       [0.91212121],
       [0.93333333],
       [0.93333333],
       [0.83863636],
       [0.79090909],
       [0.89459459],
       [0.92424242],
       [0.86942675],
       [0.925     ],
       [0.82272727],
       [0.81818182],
       [0.83378378],
       [0.83918919],
       [0.89621212],
       [0.889

### Remove rows from the matrix, such that a_ij*matrix_solution_minimum>b_

In [268]:
list_of_correct_rows = []
list_of_not_correct_rows = []
for i in range(len(wektor_po_wymnozeniu)):
    if b_to_check[i,0] > data[header_to_compare][i]:
        list_of_correct_rows.append(i)
    else: list_of_not_correct_rows.append(i)
# Amount of incorrect and correct rows
"Correct: ", len(list_of_correct_rows),"Incorrect:", len(list_of_not_correct_rows)

('Correct: ', 140, 'Incorrect:', 5)

In [269]:
data_correct = data.copy()
data_correct= data_correct.drop(axis='index', index=list_of_not_correct_rows)
data_correct

,Explained by: GDP per capita,Explained by: Social support,Explained by: Healthy life expectancy,Explained by: Freedom to make life choices,Explained by: Generosity,Explained by: Perceptions of corruption,Happiness score
2,0.876415,1.000000,0.852442,0.970270,0.576923,0.325383,0.951265
5,1.000000,0.875000,0.838641,0.945946,0.256410,0.660988,0.923020
6,0.869172,0.912121,0.852442,0.978378,0.465812,0.872232,0.919328
7,0.904029,0.938636,0.834395,0.983784,0.463675,0.807496,0.915821
8,0.826618,0.925000,0.868365,0.767568,0.331197,0.243612,0.915636
...,...,...,...,...,...,...,...
140,0.379810,0.642424,0.000000,0.566216,0.162393,0.030664,0.204541
141,0.680398,0.617424,0.297240,0.771622,0.025641,0.173765,0.196972
142,0.355364,0.100758,0.490446,0.839189,0.399573,0.926746,0.159498
143,0.428701,0.522727,0.286624,0.444595,0.226496,0.178876,0.109101


In [270]:
data_not_correct = data.copy()
data_not_correct = data_not_correct.drop(axis='index', index=list_of_correct_rows)
data_not_correct

,Explained by: GDP per capita,Explained by: Social support,Explained by: Healthy life expectancy,Explained by: Freedom to make life choices,Explained by: Generosity,Explained by: Perceptions of corruption,Happiness score
0,0.856496,0.953030,0.822718,0.994595,0.232906,0.909710,1.000000
1,0.884110,0.941667,0.824841,0.971622,0.401709,0.906303,0.965848
3,0.917157,0.928788,0.872611,0.914865,0.314103,0.785349,0.942957
4,0.880489,0.913636,0.835456,0.879730,0.579060,0.713799,0.925051
108,0.383884,0.428788,0.291932,0.451351,0.457265,0.197615,0.459110


In [271]:
data_correct.to_csv(lines[1]+"\\"+"solution_correct.csv")
data_not_correct.to_csv(lines[1]+"\\"+"solution_incorrect.csv")

In [262]:
lines[1]

'.'